In [34]:
%load_ext autoreload
%autoreload 2

In [31]:
from timm.models.vision_transformer import VisionTransformer
import torch

model = VisionTransformer(img_size=32, patch_size=4, num_classes=0, embed_dim=128, depth=4, num_heads=8, mlp_ratio=4)
#print(model)

x = torch.randn(32, 3, 32, 32)
model(x).shape

torch.Size([32, 128])

In [3]:
from timm.layers.patch_dropout import PatchDropout
from vitssm.models.modules import LearnablePositionalEncoding
import torch

drop = PatchDropout(num_prefix_tokens=0)
pos_enc = LearnablePositionalEncoding(16**2, 128, p_dropout=0.5)

x = torch.ones((32, 32**2, 128))

x = pos_enc(x)
x = drop(x)
print(x.shape)

torch.Size([32, 512, 128])


In [8]:
tuple(x // 16 for x in (224, 224))

(14, 14)

In [5]:
from xformers.components.multi_head_dispatch import MultiHeadDispatch
from xformers.components.attention import ScaledDotProduct, LocalAttention
import torch
from torch import nn

#att = ScaledDotProduct(causal=True, seq_len=30)
att = LocalAttention(causal=True, window_size=5)
blocks = nn.Sequential(*(MultiHeadDispatch(128, 4, att) for _ in range(4)))

x = torch.randn(32, 30, 128)

blocks

Sequential(
  (0): MultiHeadDispatch(
    (attention): LocalAttention(
      (attn_drop): Dropout(p=0.0, inplace=False)
    )
    (in_proj_container): InputProjection(
      (q_proj): Linear(in_features=128, out_features=128, bias=True)
      (k_proj): Linear(in_features=128, out_features=128, bias=True)
      (v_proj): Linear(in_features=128, out_features=128, bias=True)
    )
    (resid_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=128, out_features=128, bias=True)
  )
  (1): MultiHeadDispatch(
    (attention): LocalAttention(
      (attn_drop): Dropout(p=0.0, inplace=False)
    )
    (in_proj_container): InputProjection(
      (q_proj): Linear(in_features=128, out_features=128, bias=True)
      (k_proj): Linear(in_features=128, out_features=128, bias=True)
      (v_proj): Linear(in_features=128, out_features=128, bias=True)
    )
    (resid_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=128, out_features=128, bias=True)
  )
  (2): MultiHead

In [6]:
from xformers.components.attention import ScaledDotProduct
from xformers.ops.fmha.attn_bias import LowerTriangularMask
import torch
from torch import nn

att = ScaledDotProduct(seq_len=30, to_seq_len=40)
q_proj = nn.Linear(128, 128)
k_proj = nn.Linear(128, 128)
v_proj = nn.Linear(128, 128)

q = torch.ones((32, 30, 128))
k = torch.ones((32, 30, 128))
v = torch.ones((32, 40, 128))
mask = LowerTriangularMask()

att(q_proj(q), k_proj(k), v_proj(v), mask=mask).shape

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [32, 30] but got: [32, 40].

In [22]:
from vitssm.models.next_frame_prediction import LatentNextFramePrediction
from xformers.components.positional_embedding import SinePositionalEmbedding
import xformers
from torch import nn
import torch


patchify = nn.Conv2d(3, 128, kernel_size=4, stride=4)
pos_enc = SinePositionalEmbedding(128)

x = torch.zeros((32, 30, 32, 32, 3))
b, t, h, w, c = x.shape
x = rearrange(x, "b t h w c -> (b t) c h w")
print(x.shape)
x = patchify(x)
print(x.shape)
x = rearrange(x, "(b t) e h w -> (b t) (h w) e", b=b, t=t)
print(x.shape)
x = pos_enc(x)

torch.Size([960, 3, 32, 32])
torch.Size([960, 128, 8, 8])
torch.Size([960, 64, 128])


In [32]:
from xformers.components.positional_embedding import SinePositionalEmbedding
from einops import rearrange

pos_enc = SinePositionalEmbedding(3)

x1 = torch.zeros((1, 1, 4, 4, 3))
x2 = torch.zeros((1, 1, 8, 8, 3))
b1, t1, h1, w1, c1 = x1.shape
b2, t2, h2, w2, c2 = x2.shape

x1 = rearrange(x1, "b t h w c -> b (t h w) c")
x1 = pos_enc(x1)
#x1 = rearrange(x1, "b (t h w) c -> (b t) h w c", b=b1, t=t1, h=h1, w=w1)

x2 = rearrange(x2, "b t h w c -> b (t h w) c")
x2 = pos_enc(x2)
#x2 = rearrange(x2, "b (t h w) c -> (b t) h w c", b=b2, t=t2, h=h2, w=w2)


tensor([[[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]]])


In [1]:
from vitssm.models.next_frame_prediction import LatentNextFramePrediction
import torch

model = LatentNextFramePrediction(frame_in_size=(32, 32), frame_out_size=(32, 32), patch_size=4, latent_dim=32, n_heads=2, n_frames=10)
x = torch.randn(32, 10, 32, 32, 3)
out = model(x)
print(out.shape)

/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE
/home/david/micromamba/envs/ViTSSM/lib/python3

torch.Size([320, 1024, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 1024, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 1024, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 1024, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([32, 10, 32, 32, 32])


In [3]:
from einops import rearrange

x_next_frame = torch.randn(9, 4*4, 32*32*3)
t, h, w = 3, 32, 32

x_next_frame = rearrange(
    x_next_frame,
    "(b t) (hp wp) (h w c) -> b t h w c",
    t=t,
    h=h,
    w=w,
)
print(x_next_frame.shape)

EinopsError:  Error while processing rearrange-reduction pattern "(b t) (hp wp) (h w c) -> b t h w c".
 Input tensor shape: torch.Size([9, 16, 3072]). Additional info: {'t': 3, 'h': 32, 'w': 32}.
 Identifiers only on one side of expression (should be on both): {'hp', 'wp'}

In [3]:
from torch import nn
nn.activation

AttributeError: module 'torch.nn' has no attribute 'activation'

In [10]:
from math import sqrt
sqrt()

14.966629547095765

In [3]:
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("run_configs") / "base_config.yml")
config = OmegaConf.load(Path("run_configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

In [1]:
from torchvision.datasets import MovingMNIST
from vitssm.data.datasets import NextFrameDataset


mnist = MovingMNIST(root="data", split="train", split_ratio=10, download=False)
data = NextFrameDataset(mnist)


(tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         ...,
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
      

In [2]:
print(data[0][0].shape, data[0][1].shape)

torch.Size([9, 1, 64, 64]) torch.Size([9, 1, 64, 64])


In [5]:
import xformers
import torch
torch.cuda.is_available()

True

In [6]:
from vitssm.engine.tasks import ActionRecognitionEngine
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
engine = ActionRecognitionEngine(model, conf)

ModuleNotFoundError: No module named 'addict'

In [ ]:
from vitssm.utils.metrics import MetricContainer
from torchmetrics import Accuracy, Precision, Recall
import torch

x = torch.randn(100, 100)
y = torch.randint(0, 2, (100, 100))
metric_col(x, y)

In [ ]:
from vitssm.engine import build_optimizer
from addict import Dict
from torch import nn

config = Dict({
    "optimization": {
        "optimizer": {
            "name": "Adam",
            "args": {
                "lr": 0.001,
                "weight_decay": 0.0
            }
        }
    }    
})

model = nn.Linear(10, 10)
build_optimizer(model, config)

In [ ]:
from vitssm.data import build_dataset
from addict import Dict
import kappaconfig as kc
import os
from pathlib import Path

config = kc.from_file_uri(Path("run_configs") / "base_config.yml")
print(config.keys())
res = kc.DefaultResolver()
config = Dict(res.resolve(config))

dataset = build_dataset(config)

In [ ]:
dataset[1000][2]

In [ ]:
print(os.getcwd())
os.chdir()

In [ ]:
dataset

In [ ]:
train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3).shape

In [ ]:
from torchvision.io import write_video

write_video("test.mp4", train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3), fps=10)